# 5.2.3 Representación TF-IDF

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings("ignore")

En una matriz documento-término, las cantidades representan la ocurrencia de un token en cada uno de los documentos.

Las palabras como artículos, verbos ser/estar, conectores, etc. son palabras muy comunes en los textos (tienen frecuencias altas en una matriz documento-término) y tienen poca utilidad para extraer información valiosa de un documento. Adicionalmente, distorcionan y obscurecen términos interesantes que serían de mucha más utilidad.

La representación TF-IDF (term-frequency inverse document-frequency) recomputa los valores de la matriz como:

$$ tf-idf(t,d) = tf(t,d) x idf(t) $$

- $t$ representa el término.
- $d$ representa el documento.

La transformación también pondera la frecuencia de cada token respecto a su frecuencia en el documento y la cantidad de tokens en dicho documento.

Para explicar esta representación considere la siguiente matriz documento-termino:

In [ ]:
#
# tf: term-frequency
#
tf = np.array(
    [
        [3, 0, 1],
        [2, 0, 0],
        [3, 0, 0],
        [4, 0, 0],
        [3, 2, 0],
        [3, 0, 2],
    ]
)

tf

Ahora se crea el transformador:

In [ ]:
transformer = TfidfTransformer(
    # -------------------------------------------------------------
    # Each output row will have unit norm.
    # "l1", "l2"
    norm="l1",
    # -------------------------------------------------------------
    # Enable inverse-document-frequency reweighting.
    use_idf=True,
    # -------------------------------------------------------------
    # Smooth idf weights by adding one to document frequencies, as
    # if an extra document was seen containing every term in the
    # collection exactly once. Prevents zero divisions.
    smooth_idf=False,
    # -------------------------------------------------------------
    # Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
    sublinear_tf=False,
)

transformer.fit_transform(tf).toarray()

## 5.2.3.1 Caso 1

* norm="l1"

* use_idf=False

In [ ]:
#
# term-frequency
#
tf

In [ ]:
#
# norma "l1" de cada fila
#
row_norm = np.tile(tf.sum(axis=1).reshape(-1, 1), (1, 3))
row_norm

In [ ]:
tf / row_norm

In [ ]:
#
# Verificación
#
TfidfTransformer(
    norm="l1",
    use_idf=False,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

## 5.2.3.2 Caso 2

* norm="l2"

* use_idf=False

In [ ]:
#
# term-frequency
#
tf

In [ ]:
#
# norma "l2" de cada fila
#
row_norm = np.tile(np.sqrt(np.power(tf, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

In [ ]:
tf / row_norm

In [ ]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=False,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

## 5.2.3.3 Caso 3

* norm="l2"

* use_idf=True

* smooth_idf=False

* sublinear_tf=False

In [ ]:
#
# número de documentos = 6
#
n = tf.shape[0]

In [ ]:
#
# Cuenta los documentos en que aparece un término
#
df = np.where(tf > 0, 1, 0)
df = df.sum(axis=0)
df

In [ ]:
#
# Computa idf(t). Para smooth_idf=False
#
#                  n
#   idf(t) = log ---- + 1
#                 df
#
idf = np.log(n / df) + 1
idf = np.tile(idf, (6, 1))
idf

In [ ]:
tf_idf_raw = tf * idf
tf_idf_raw

In [ ]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

In [ ]:
tf_idf = tf_idf_raw / row_norm
tf_idf

In [ ]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

## 5.2.3.4 Caso 4

* norm="l2"

* use_idf=True

* smooth_idf=True

* sublinear_tf=False

In [ ]:
#
# Computa idf(t). Para smooth_idf=True. Equivale
# a un documento que tiene todos los terminos
#
#                  1+n
#   idf(t) = log ------ + 1
#                 1+df
#
idf = np.log((1 + n) / (1 + df)) + 1
idf = np.tile(idf, (6, 1))
idf

In [ ]:
tf_idf_raw = tf * idf
tf_idf_raw

In [ ]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

In [ ]:
tf_idf = tf_idf_raw / row_norm
tf_idf

In [ ]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
).fit_transform(tf).toarray()

## 5.2.3.5 Caso 5

* norm="l2"

* use_idf=True

* smooth_idf=True

* sublinear_tf=True

In [ ]:
#
# Computa idf(t). Para smooth_idf=True. Equivale
# a un documento que tiene todos los terminos
#
#                  1+n
#   idf(t) = log ------ + 1
#                 1+df
#
idf = np.log((1 + n) / (1 + df)) + 1
idf = np.tile(idf, (6, 1))
idf

In [ ]:
#
# Cuando sublinear_tf=True, reemplaza tf por 1 + log(tf)
#
mylog = lambda x: 1 + np.log(x) if x > 0 else 0

tf_idf_raw = np.vectorize(mylog)(tf) * idf
tf_idf_raw

In [ ]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

In [ ]:
tf_idf = tf_idf_raw / row_norm
tf_idf

In [ ]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
).fit_transform(tf).toarray()

In [ ]:
print('ok_')